In [ ]:
# Colab setup: install libs and set Kaggle credentials
!pip install -q kaggle
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q pandas pillow tqdm scikit-learn matplotlib efficientnet_pytorch

# (Optional) show versions
import sys, torch, torchvision
print("python", sys.version.split()[0])
print("torch", torch.__version__, "torchvision", torchvision.__version__)

# Set Kaggle credentials (replace with your values if needed)
import os
os.environ['KAGGLE_USERNAME'] = "nikhilsanga1206"   # replace if different
os.environ['KAGGLE_KEY'] = "16c0c673e500366baf8a2262a8494565"  # replace if different


  Preparing metadata (setup.py) ... done
python 3.12.12
torch 2.8.0+cu126 torchvision 0.23.0+cu126


In [ ]:
# Download dataset (this will save a zip in the current working dir)
!kaggle datasets download -d khanfashee/nih-chest-x-ray-14-224x224-resized -p /content --unzip
# If the dataset is already a folder after unzip, list it
!ls -lah /content | sed -n '1,120p'


Dataset URL: https://www.kaggle.com/datasets/khanfashee/nih-chest-x-ray-14-224x224-resized
License(s): CC0-1.0
 99% 2.28G/2.30G [00:28<00:00, 91.3MB/s]
100% 2.30G/2.30G [00:32<00:00, 76.6MB/s]
total 38M
drwxr-xr-x 1 root root 4.0K Nov 11 13:59 .
drwxr-xr-x 1 root root 4.0K Nov 11 13:55 ..
-rw-r--r-- 1 root root  91K Nov 11 13:58 BBox_List_2017_Official_NIH.csv
drwxr-xr-x 4 root root 4.0K Nov  7 14:30 .config
-rw-r--r-- 1 root root 8.3M Nov 11 13:58 Data_Entry_2017.csv
drwxr-xr-x 3 root root 4.0K Nov 11 13:58 images-224
-rw-r--r-- 1 root root  28M Nov 11 13:59 pretrained_model.h5
drwxr-xr-x 1 root root 4.0K Nov  7 14:30 sample_data
-rw-r--r-- 1 root root 425K Nov 11 13:59 test_list_NIH.txt
-rw-r--r-- 1 root root 1.5M Nov 11 13:59 train_val_list_NIH.txt


In [ ]:
import pandas as pd
meta = pd.read_csv('/content/Data_Entry_2017.csv')
print("Columns:", meta.columns.tolist())
print("Total rows:", len(meta))
print("Unique images:", meta['Image Index'].nunique())
print("Sample rows:")
display(meta.head())

# If bbox CSV exists
import os
bbox_path = '/content/BBox_List_2017_Official_NIH.csv'
if os.path.exists(bbox_path):
    bbox = pd.read_csv(bbox_path)
    print("BBox rows:", len(bbox))
    display(bbox.head())
else:
    print("BBox CSV not found at", bbox_path)


Columns: ['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID', 'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11']
Total rows: 112120
Unique images: 112120
Sample rows:


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,058Y,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,058Y,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,058Y,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,081Y,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,081Y,F,PA,2582,2991,0.143,0.143,NaN


BBox rows: 984


,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,NaN,NaN,NaN
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,NaN,NaN,NaN
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,NaN,NaN,NaN
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,NaN,NaN,NaN
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,NaN,NaN,NaN


In [ ]:
import pandas as pd

bbox_path = "/content/BBox_List_2017_Official_NIH.csv"  # adjust if different
bbox_df = pd.read_csv(bbox_path)
print("🔍 Actual columns in your BBox file:")
for c in bbox_df.columns:
    print("-", c)


🔍 Actual columns in your BBox file:
- Image Index
- Finding Label
- Bbox [x
- y
- w
- h]
- Unnamed: 6
- Unnamed: 7
- Unnamed: 8


In [ ]:
import json
import pandas as pd

meta_path = "/content/Data_Entry_2017.csv"
bbox_path = "/content/BBox_List_2017_Official_NIH.csv"

meta_df = pd.read_csv(meta_path)
bbox_df = pd.read_csv(bbox_path)

# --- Normalize column names ---
bbox_df.columns = [c.strip().replace("[", "").replace("]", "").replace(" ", "_").lower() for c in bbox_df.columns]
meta_df.columns = [c.strip().replace("[", "").replace("]", "").replace(" ", "_").lower() for c in meta_df.columns]

print("✅ Cleaned BBox columns:", bbox_df.columns.tolist())

# --- Pick coordinate columns manually (we know them now) ---
x_col = "bbox_x"
y_col = "y"
w_col = "w"
h_col = "h"

# --- Build COCO structure ---
coco = {"images": [], "annotations": [], "categories": []}
categories = {label: idx + 1 for idx, label in enumerate(bbox_df["finding_label"].unique())}
for name, idx in categories.items():
    coco["categories"].append({"id": idx, "name": name})

img_id_map = {}
img_counter = 1
ann_id = 1

for _, row in bbox_df.iterrows():
    file_name = row["image_index"]
    label = row["finding_label"]

    # Get bbox values safely
    try:
        x = float(row[x_col])
        y = float(row[y_col])
        w = float(row[w_col])
        h = float(row[h_col])
    except Exception as e:
        print(f"⚠️ Skipping {file_name}: invalid bbox values ({e})")
        continue

    # Add image if not already added
    if file_name not in img_id_map:
        meta_row = meta_df[meta_df["image_index"] == file_name]
        if "originalimagewidth" in meta_row.columns:
            width = int(meta_row["originalimagewidth"].iloc[0])
        else:
            width = 224
        if "originalimageheight" in meta_row.columns:
            height = int(meta_row["originalimageheight"].iloc[0])
        else:
            height = 224

        coco["images"].append({
            "id": img_counter,
            "file_name": file_name,
            "width": width,
            "height": height
        })
        img_id_map[file_name] = img_counter
        img_counter += 1

    # Add annotation
    coco["annotations"].append({
        "id": ann_id,
        "image_id": img_id_map[file_name],
        "category_id": categories[label],
        "bbox": [x, y, w, h],
        "area": w * h,
        "iscrowd": 0
    })
    ann_id += 1

# --- Save COCO file ---
out_path = "/content/coco_chestxray.json"
with open(out_path, "w") as f:
    json.dump(coco, f, indent=4)

print(f"✅ COCO JSON saved to: {out_path}")
print(f"📸 Total images: {len(coco['images'])}, annotations: {len(coco['annotations'])}")


✅ Cleaned BBox columns: ['image_index', 'finding_label', 'bbox_x', 'y', 'w', 'h', 'unnamed:_6', 'unnamed:_7', 'unnamed:_8']
✅ COCO JSON saved to: /content/coco_chestxray.json
📸 Total images: 880, annotations: 984


In [ ]:
import pandas as pd

CSV_PATH = "/content/Data_Entry_2017.csv"
df = pd.read_csv(CSV_PATH)

print("Columns in dataset:")
print(df.columns.tolist())
print("\nTotal records:", len(df))
print("Unique images:", df["Image Index"].nunique())

# Convert 'Finding Labels' column into list of labels
df["Finding Labels"] = df["Finding Labels"].fillna("No Finding")
df["labels_list"] = df["Finding Labels"].apply(lambda x: x.split("|"))
print("\nSample:")
print(df[["Image Index", "labels_list"]].head(5))


Columns in dataset:
['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID', 'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11']

Total records: 112120
Unique images: 112120

Sample:
        Image Index                labels_list
0  00000001_000.png             [Cardiomegaly]
1  00000001_001.png  [Cardiomegaly, Emphysema]
2  00000001_002.png   [Cardiomegaly, Effusion]
3  00000002_000.png               [No Finding]
4  00000003_000.png                   [Hernia]


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer
import torchvision.transforms as T
from PIL import Image
import numpy as np
import random
import os

IMG_DIR = "/content/images-224/images-224"  # adjust if needed
IMG_SIZE = 224
BATCH_SIZE = 32

# Create label encoder
mlb = MultiLabelBinarizer()
mlb.fit(df["labels_list"])
CLASSES = mlb.classes_
NUM_CLASSES = len(CLASSES)
print("✅ Number of labels:", NUM_CLASSES)
print("Some labels:", CLASSES[:10])

# Create dataset class
class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, img_dir, mlb, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.mlb = mlb
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["Image Index"])
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        labels = self.mlb.transform([row["labels_list"]])[0]
        return img, torch.tensor(labels, dtype=torch.float32)

# Split train/val
random.seed(42)
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

# Transforms
train_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(8),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Dataloaders
train_loader = DataLoader(ChestXrayDataset(train_df, IMG_DIR, mlb, train_tfms),
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(ChestXrayDataset(val_df, IMG_DIR, mlb, val_tfms),
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("Train batches:", len(train_loader), "Val batches:", len(val_loader))


✅ Number of labels: 15
Some labels: ['Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Emphysema' 'Fibrosis' 'Hernia' 'Infiltration' 'Mass']
Train batches: 2803 Val batches: 701


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Model setup (EfficientNet-B0)
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)  # new API
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, NUM_CLASSES)
model = model.to(device)

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

# LR scheduler (no verbose for new versions)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)


Using device: cuda
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 224MB/s]


In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np
import torch
import os

EPOCHS = 10  # 🔢 you can increase later (e.g. 10–20)
best_f1 = 0.0
OUT_DIR = "/content/checkpoints"
os.makedirs(OUT_DIR, exist_ok=True)

def train_one_epoch(model, loader):
    model.train()
    total_loss = 0.0
    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outs = model(imgs)
        loss = criterion(outs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
    return total_loss / len(loader.dataset)

@torch.no_grad()
def validate(model, loader):
    model.eval()
    total_loss = 0.0
    preds_all, labels_all = [], []
    for imgs, labels in tqdm(loader, desc="Val", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        outs = model(imgs)
        loss = criterion(outs, labels)
        total_loss += loss.item() * imgs.size(0)
        probs = torch.sigmoid(outs).cpu().numpy()
        preds = (probs >= 0.5).astype(int)
        preds_all.append(preds)
        labels_all.append(labels.cpu().numpy())
    preds_all = np.vstack(preds_all)
    labels_all = np.vstack(labels_all)
    f1 = f1_score(labels_all, preds_all, average="macro", zero_division=0)
    return total_loss / len(loader.dataset), f1

for epoch in range(1, EPOCHS + 1):
    print(f"\n🧩 Epoch {epoch}/{EPOCHS}")
    train_loss = train_one_epoch(model, train_loader)
    val_loss, val_f1 = validate(model, val_loader)
    scheduler.step(val_loss)
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")
    print(f"📉 Current LR: {optimizer.param_groups[0]['lr']:.6e}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), f"{OUT_DIR}/classifier_best.pth")
        torch.save(model, f"{OUT_DIR}/classifier_best.pkl")
        print("✅ Saved best model (pth + pkl)")

print("🎉 Training complete! Best F1:", best_f1)



🧩 Epoch 1/10


Train Loss: 0.1952 | Val Loss: 0.1787 | Val F1: 0.1491
📉 Current LR: 1.000000e-04
✅ Saved best model (pth + pkl)

🧩 Epoch 2/10


Train:   0%|          | 0/2803 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Train Loss: 0.1789 | Val Loss: 0.1749 | Val F1: 0.1667
📉 Current LR: 1.000000e-04
✅ Saved best model (pth + pkl)

🧩 Epoch 3/10


Train:   0%|          | 0/2803 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...

Train Loss: 0.1569 | Val Loss: 0.1760 | Val F1: 0.2550
📉 Current LR: 1.000000e-04
✅ Saved best model (pth + pkl)

🧩 Epoch 9/10


Train:   0%|          | 0/2803 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Train Loss: 0.1530 | Val Loss: 0.1786 | Val F1: 0.2701
📉 Current LR: 5.000000e-05
✅ Saved best model (pth + pkl)

🧩 Epoch 10/10


Train:   0%|          | 0/2803 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Train Loss: 0.1441 | Val Loss: 0.1835 | Val F1: 0.2793
📉 Current LR: 5.000000e-05
✅ Saved best model (pth + pkl)
🎉 Training complete! Best F1: 0.2793493742772772


In [ ]:
from google.colab import files
files.download("/content/checkpoints/classifier_best.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 32.6 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim


In [ ]:
def objective(trial):
    # 🎯 1️⃣ Suggest hyperparameters
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    optimizer_name = trial.suggest_categorical('optimizer', ['AdamW', 'SGD'])
    backbone = trial.suggest_categorical('backbone', ['efficientnet_b0', 'resnet50'])

    # 🎯 2️⃣ Build model dynamically
    if backbone == 'efficientnet_b0':
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        in_feats = model.classifier[1].in_features
        model.classifier[1] = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_feats, NUM_CLASSES)
        )
    else:
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        in_feats = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_feats, NUM_CLASSES)
        )

    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()

    # 🎯 3️⃣ Optimizer choice
    if optimizer_name == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.9)

    # 🎯 4️⃣ Train briefly (for speed)
    model.train()
    for epoch in range(5):  # only 2 epochs per trial for quick evaluation
        for imgs, labels in tqdm(train_loader, desc=f"Trial {trial.number} Epoch {epoch+1}/2", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outs = model(imgs)
            loss = criterion(outs, labels)
            loss.backward()
            optimizer.step()

    # 🎯 5️⃣ Evaluate on validation
    model.eval()
    preds_all, labels_all = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outs = model(imgs)
            probs = torch.sigmoid(outs).cpu().numpy()
            preds = (probs >= 0.5).astype(int)
            preds_all.append(preds)
            labels_all.append(labels.cpu().numpy())
    preds_all = np.vstack(preds_all)
    labels_all = np.vstack(labels_all)
    f1 = f1_score(labels_all, preds_all, average='macro', zero_division=0)

    print(f"Trial {trial.number} completed with F1: {f1:.4f}")
    return f1


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)   # increase n_trials for deeper tuning

print("✅ Best trial found:")
print(study.best_trial.params)


In [ ]:
best = study.best_trial.params

# Build best model again
if best['backbone'] == 'efficientnet_b0':
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    in_feats = model.classifier[1].in_features
    model.classifier[1] = nn.Sequential(
        nn.Dropout(best['dropout_rate']),
        nn.Linear(in_feats, NUM_CLASSES)
    )
else:
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    in_feats = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(best['dropout_rate']),
        nn.Linear(in_feats, NUM_CLASSES)
    )

model = model.to(device)
criterion = nn.BCEWithLogitsLoss()

if best['optimizer'] == 'AdamW':
    optimizer = optim.AdamW(model.parameters(), lr=best['lr'], weight_decay=best['weight_decay'])
else:
    optimizer = optim.SGD(model.parameters(), lr=best['lr'], weight_decay=best['weight_decay'], momentum=0.9)

print("✅ Training final model with tuned parameters:", best)


✅ Training final model with tuned parameters: {'lr': 1.671869569920371e-05, 'weight_decay': 0.0002125014538846767, 'dropout_rate': 0.2157050777375356, 'optimizer': 'SGD', 'backbone': 'efficientnet_b0'}


In [ ]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_parallel_coordinate(study)


In [ ]:
import torch, os

OUT_DIR = "/content/checkpoints"
os.makedirs(OUT_DIR, exist_ok=True)

# Save both formats
torch.save(model.state_dict(), f"{OUT_DIR}/classifier_tuned_best.pth")
torch.save(model, f"{OUT_DIR}/classifier_tuned_best.pkl")

print("✅ Saved tuned model successfully:")
print(f"- Weights: {OUT_DIR}/classifier_tuned_best.pth")
print(f"- Full model: {OUT_DIR}/classifier_tuned_best.pkl")


✅ Saved tuned model successfully:
- Weights: /content/checkpoints/classifier_tuned_best.pth
- Full model: /content/checkpoints/classifier_tuned_best.pkl


In [ ]:
from google.colab import files
files.download("/content/checkpoints/classifier_tuned_best.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
from tqdm import tqdm
import numpy as np
from sklearn.metrics import f1_score
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import os

# --- Load best params from Optuna ---
best_params = {
    'lr': 1.671869569920371e-05,
    'weight_decay': 0.0002125014538846767,
    'dropout_rate': 0.2157050777375356,
    'optimizer': 'SGD',
    'backbone': 'efficientnet_b0'
}

# --- Build model ---
if best_params['backbone'] == 'efficientnet_b0':
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    in_feats = model.classifier[1].in_features
    model.classifier[1] = nn.Sequential(
        nn.Dropout(best_params['dropout_rate']),
        nn.Linear(in_feats, NUM_CLASSES)
    )
else:
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    in_feats = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(best_params['dropout_rate']),
        nn.Linear(in_feats, NUM_CLASSES)
    )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()

if best_params['optimizer'] == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'], momentum=0.9)
else:
    optimizer = optim.AdamW(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])

# --- Train the tuned model fully ---
EPOCHS = 10
best_f1 = 0.0
OUT_DIR = "/content/checkpoints"
os.makedirs(OUT_DIR, exist_ok=True)

def train_one_epoch(model, loader):
    model.train()
    total_loss = 0.0
    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outs = model(imgs)
        loss = criterion(outs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
    return total_loss / len(loader.dataset)

@torch.no_grad()
def validate(model, loader):
    model.eval()
    total_loss = 0.0
    preds_all, labels_all = [], []
    for imgs, labels in tqdm(loader, desc="Val", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        outs = model(imgs)
        loss = criterion(outs, labels)
        total_loss += loss.item() * imgs.size(0)
        probs = torch.sigmoid(outs).cpu().numpy()
        preds = (probs >= 0.5).astype(int)
        preds_all.append(preds)
        labels_all.append(labels.cpu().numpy())
    preds_all = np.vstack(preds_all)
    labels_all = np.vstack(labels_all)
    f1 = f1_score(labels_all, preds_all, average="macro", zero_division=0)
    return total_loss / len(loader.dataset), f1

for epoch in range(1, EPOCHS + 1):
    print(f"\n🧩 Epoch {epoch}/{EPOCHS}")
    train_loss = train_one_epoch(model, train_loader)
    val_loss, val_f1 = validate(model, val_loader)
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), f"{OUT_DIR}/classifier_tuned_best.pth")
        torch.save(model, f"{OUT_DIR}/classifier_tuned_best.pkl")
        print("✅ Saved tuned best model (pth + pkl)")

print("🎯 Final training complete! Best tuned F1:", best_f1)



🧩 Epoch 1/10


Train:   0%|          | 0/2803 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Train Loss: 0.6150 | Val Loss: 0.5356 | Val F1: 0.0549
✅ Saved tuned best model (pth + pkl)

🧩 Epoch 2/10


Train:   0%|          | 0/2803 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



KeyboardInterrupt: 

In [ ]:
# Build COCO json from NIH bbox + metadata (optional, used later for detection)
import json
from collections import defaultdict
import pandas as pd
meta = pd.read_csv('/content/Data_Entry_2017.csv')
bbox_path = '/content/BBox_List_2017_Official_NIH.csv'
if os.path.exists(bbox_path):
    bbox = pd.read_csv(bbox_path)
    # cleanup column names if needed
    bbox.columns = [c.strip() for c in bbox.columns]
    meta.columns = [c.strip() for c in meta.columns]
    # categories
    cats = sorted(bbox['Finding Label'].unique().tolist())
    cat_map = {name: i+1 for i, name in enumerate(cats)}
    coco = {"images": [], "annotations": [], "categories": []}
    for name, idx in cat_map.items():
        coco["categories"].append({"id": idx, "name": name})
    img_map = {}
    img_id = 1
    ann_id = 1
    for _, row in meta.iterrows():
        fname = row['Image Index']
        w = int(row.get('OriginalImageWidth', 224))
        h = int(row.get('OriginalImageHeight', 224)) if 'OriginalImageHeight' in row else int(row.get('OriginalImageHeight', 224) if 'OriginalImageHeight' in row else 224)
        coco['images'].append({"id": img_id, "file_name": fname, "width": w, "height": h})
        img_map[fname] = img_id
        img_id += 1
    # annotations
    for _, r in bbox.iterrows():
        fname = r['Image Index']
        if fname not in img_map:
            continue
        coco['annotations'].append({
            "id": ann_id,
            "image_id": img_map[fname],
            "category_id": cat_map[r['Finding Label']],
            "bbox": [int(r['Bbox x']), int(r['y']), int(r['w']), int(r['h'])],
            "area": int(r['w']) * int(r['h']),
            "iscrowd": 0
        })
        ann_id += 1
    with open('/content/coco_chestxray.json', 'w') as f:
        json.dump(coco, f)
    print("Saved /content/coco_chestxray.json with", len(coco['images']), "images and", len(coco['annotations']), "annotations.")
else:
    print("BBox file not found, skipping COCO build.")


KeyError: 'Bbox x'

In [ ]:
# Classification dataset + dataloader setup
import os, random
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import numpy as np
import pandas as pd

CSV_PATH = '/content/Data_Entry_2017.csv'
IMG_DIR = '/content/images-224/images-224'   # adjust if your images are elsewhere

# 1) load csv and build items (filename, [labels])
df = pd.read_csv(CSV_PATH)
# ensure correct column names
if 'Image Index' not in df.columns:
    raise RuntimeError("CSV doesn't have 'Image Index' column")
if 'Finding Labels' not in df.columns and 'Finding Label' in df.columns:
    df = df.rename(columns={'Finding Label':'Finding Labels'})
df['Finding Labels'] = df['Finding Labels'].fillna('No Finding')
df['labels_list'] = df['Finding Labels'].apply(lambda s: [x for x in str(s).split('|') if x!=''])
items = list(zip(df['Image Index'].tolist(), df['labels_list'].tolist()))
print("Total images:", len(items))

# build MultiLabelBinarizer
all_labels = [lbl for _, lbl in items]
mlb = MultiLabelBinarizer()
mlb.fit(all_labels)
classes = list(mlb.classes_)
print("Classes (N):", len(classes), classes[:20])

# dataset class
class MultiLabelChestXrayDataset(Dataset):
    def __init__(self, items, img_dir, mlb, transform=None):
        self.items = items
        self.img_dir = img_dir
        self.mlb = mlb
        self.transform = transform

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        fname, labels = self.items[idx]
        path = os.path.join(self.img_dir, fname)
        image = Image.open(path).convert("RGB")
        if self.transform: image = self.transform(image)
        y = self.mlb.transform([labels])[0].astype(np.float32)
        return image, torch.tensor(y, dtype=torch.float32)

# transforms
IMG_SIZE = 224
train_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(8),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# train/val split
random.seed(42)
idxs = list(range(len(items)))
random.shuffle(idxs)
split = int(0.8 * len(idxs))
train_items = [items[i] for i in idxs[:split]]
val_items = [items[i] for i in idxs[split:]]
print("Train images:", len(train_items), "Val images:", len(val_items))

# datasets + dataloaders
BATCH_SIZE = 32
train_ds = MultiLabelChestXrayDataset(train_items, IMG_DIR, mlb, train_transform)
val_ds = MultiLabelChestXrayDataset(val_items, IMG_DIR, mlb, val_transform)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
import torch.nn as nn
import torchvision.models as models
import torch

NUM_CLASSES = len(classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

BACKBONE = 'efficientnet'  # 'efficientnet' or 'resnet'
if BACKBONE == 'efficientnet':
    model = models.efficientnet_b0(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
elif BACKBONE == 'resnet':
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
else:
    raise ValueError("select backbone")
model = model.to(device)
print(model)
